In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.5 Flash > 멀티모달 모드 소개

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgetting-started%2Fintro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_5_flash.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| 저자 |
| --- |
| [Eric Dong](https://github.com/gericdong) |
| [Holt Skinner](https://github.com/holtskinner) |
| 번역/편집 |
| [T Kim](https://github.com/aimldl) |

### 목표
Gemini 2.5 Flash 모델의 멀티모달 모드를 사용하는 방법을 배웁니다.

### 멀티모달 모드
- 텍스트 프롬프트에서 텍스트 생성
- 스트리밍 텍스트 생성
- 토큰 수 계산

## 시작하기

### Python용 Google Gen AI SDK 설치

In [1]:
%pip install --upgrade --quiet google-genai

Note: you may need to restart the kernel to use updated packages.


### 노트북 환경 인증 (Colab 전용)
- Vertex AI Workbench 사용자는 생략
- Google Colab에서 이 노트북을 실행하는 경우, 아래 셀을 실행해서 환경을 인증하세요.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 라이브러리 가져오기

In [2]:
from IPython.display import HTML, Image, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    Part,
    SafetySetting,
    ThinkingConfig,
    Tool,
    ToolCodeExecution,
)

### Vertex AI용 Google Cloud 프로젝트 사용
- Google Cloud 프로젝트에서 [Vertex AI API 활성화](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)합니다.
- Google Cloud 프로젝트의 Vertex AI Workbench Instance에서 실행할 경우, 자동으로 `PROJECT_ID`와 `LOCATION`을 불러옵니다.

In [3]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

print(f"PROJECT_ID={PROJECT_ID}")
print(f"LOCATION={LOCATION}")

PROJECT_ID=vertex-workbench-notebook
LOCATION=global


In [4]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Gemini 2.5 Flash 모델의 사용

- [Gemini 2.5 Flash](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash?hl=ko)의 Model ID를 확인하려면 [여기](https://cloud.google.com/vertex-ai/generative-ai/docs/models/gemini/2-5-flash?hl=ko)를 클릭하세요. 

In [5]:
MODEL_ID = "gemini-2.5-flash-preview-05-20"  # @param {type: "string"}

## 멀티모달 프롬프트 보내기
- Gemini 2.5 Flash는 멀티모달 프롬프트를 지원하는 모델입니다.
- 다양한 소스에서 다음 데이터 유형 중 하나를 포함할 수 있습니다.

### 지원되는 데이터 유형

| 데이터 유형 | 소스 | MIME 유형 |
| :---------- | :------------------------------------ | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 텍스트 | 인라인, 로컬 파일, 일반 URL, Google Cloud Storage | `text/plain` `text/html` |
| 코드 | 인라인, 로컬 파일, 일반 URL, Google Cloud Storage | `text/plain` |
| 문서 | 로컬 파일, 일반 URL, Google Cloud Storage | `application/pdf` |
| 이미지 | 로컬 파일, 일반 URL, Google Cloud Storage | `image/jpeg` `image/png` `image/webp` |
| 비디오 | 로컬 파일, 일반 URL, Google Cloud Storage, YouTube | `video/mp4` `video/mpeg` `video/x-flv` `video/quicktime` `video/mpegps` `video/mpg` `video/webm` `video/wmv` `video/3gpp` |
| 오디오 | 로컬 파일, 일반 URL, Google Cloud Storage | `audio/aac` `audio/flac` `audio/mp3` `audio/m4a` `audio/mpeg` `audio/mpga` `audio/mp4` `audio/opus` `audio/pcm` `audio/wav` `audio/webm` |

멀티모달 사용 사례의 더 많은 예는 [이 노트북](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/intro_multimodal_use_cases.ipynb)을 참조하세요.

### 로컬 이미지 사용하기
`images/example-image-night-shot.jpg`
<img src="images/example-image-night-shot.jpg">

In [8]:
with open("images/example-image-night-shot.jpg", "rb") as f:
    image = f.read()

prompt="이 사진에 대해 설명해죠. 사진에 있는 빌딩의 이름이 뭐야?"

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        
        proPart.from_bytes(data=image, mime_type="image/png"),mpt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

이 사진은 밤에 촬영된 아름다운 도시 풍경입니다.

**사진에 대한 설명:**

*   **중앙의 빌딩:** 사진의 중심에는 매우 높고 현대적인 초고층 빌딩이 우뚝 솟아 있습니다. 이 빌딩의 상층부는 밝은 초록색 조명으로 화려하게 빛나고 있으며, 독특한 격자무늬 디자인이 특징입니다. 빌딩의 다른 층들에도 창문마다 불이 켜져 있어 밤에도 활기찬 모습을 보여줍니다.
*   **하늘:** 배경의 하늘은 어두운 남색으로, 밤임을 분명히 보여줍니다.
*   **전경의 나무:** 빌딩 앞쪽에는 여러 그루의 나무들이 늘어서 있습니다. 나무들은 가지가 위로 뻗어 있고, 밝은 빛을 받아 잎사귀들이 푸르게 빛나고 있습니다. 이 나무들은 도시의 딱딱한 건물과 대비되어 자연적인 요소를 더해줍니다.
*   **전반적인 분위기:** 전체적으로 웅장하면서도 평화로운 야경을 보여주며, 도시의 현대적인 아름다움을 잘 담아내고 있습니다.

**사진에 있는 빌딩의 이름은 바로 서울 송파구에 위치한 **롯데월드타워(Lotte World Tower)**입니다.**

### YouTube URL에서 비디오 보내기
[YouTube 비디오](https://youtu.be/LxjW4_C3_Cc?si=RIt8xZud9ZulnicC)를 활용해서 비디오 이해
- 주의: 유튜브 링크는 visibility=Public이여야 합니다. 

In [7]:
prompt="최종 게임스코어는 몇 분쯤에 나오나요?"

video = Part.from_uri(
    file_uri="https://youtu.be/LxjW4_C3_Cc?si=RIt8xZud9ZulnicC",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

최종 게임 스코어는 0분 37초부터 0분 39초까지 나옵니다.

Lotte G 9 : 4 D Doosan (2025년 6월 7일)

In [15]:
prompt="5초 이상 화면이 전환되지 않는 경우가 있는지 확인해죠."

video = Part.from_uri(
    file_uri="https://youtube.com/shorts/H4jpz54tVe0",
    mime_type="video/mp4",
)

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    #config=GenerateContentConfig(
    #    thinking_config=thinking_config,
    #),
)

display(Markdown(response.text))

네, 5초 이상 화면이 전환되지 않는 경우가 있습니다.

1.  **0:19초부터 0:39초까지 약 20초간** 장바구니 화면이 정지된 상태로 유지됩니다.
2.  **0:40초부터 0:45초까지 약 5초간** "상품을 장바구니에서 삭제할까요?"라는 알림 팝업이 정지된 상태로 유지됩니다.

In [ ]:
    config=GenerateContentConfig(
        system_instruction=system_instruction,
        #thinking_config=thinking_config,
    ),

### 일반 URL에서 오디오 보내기
이 예는 [Kubernetes Podcast]](https://kubernetespodcast.com/) 에피소드의 오디오입니다.

오디오 파일을 실행하려면 다음 링크를 클릭하세요.
- [https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3](https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3)
- 크롬 브라우저의 새 탭이 열리고, 재생할 수 있습니다.

In [9]:
#prompt="Write a summary of this podcast episode."
prompt="이 팟캐스트 에피소드를 요약한 다음, 한국어로 번역해죠."
audio_file_uri="https://traffic.libsyn.com/secure/e780d51f-f115-44a6-8252-aed9216bb521/KPOD242.mp3"
mime_type="audio/mpeg"

In [8]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_uri(
            file_uri=audio_file_uri,
            mime_type=mime_type,
        ),
        prompt,
    ],
    config=GenerateContentConfig(
        audio_timestamp=True,
        #thinking_config=thinking_config,
    ),
)

display(Markdown(response.text))

이 팟캐스트 에피소드는 KubeCon + CloudNativeCon North America 2024를 요약하며, 행사 주요 발표를 시작으로 참석자들과의 인터뷰를 통해 그들의 경험과 관찰된 트렌드를 다룹니다.

**주요 발표 및 뉴스:**
*   Cert-Manager와 Dapr가 CNCF 졸업 프로젝트가 되었습니다.
*   Istio 1.20이 출시되었으며, Ambient Mesh는 일반 가용성(GA)을 달성했습니다.
*   CNCF는 특허 괴물(patent trolls)에 맞서 싸우기 위한 Cloud Native Heroes Challenge를 시작했습니다.
*   2025년 행사 라인업에는 5개의 KubeCon + CloudNativeCon 행사, 1개의 오픈 소스 SecurityCon, 그리고 30개의 Kubernetes Community Days가 포함됩니다.
*   세 가지 새로운 클라우드 네이티브 자격증(Certified Backstage Associate, OpenTelemetry Certified Associate, Kyverno Certified Associate)이 발표되었습니다.
*   리눅스 재단은 내년부터 주요 Kubernetes 자격증(CKA, CKS, CKAD) 및 LFS 시험의 응시료를 10% 인상한다고 발표했습니다.
*   WasmCloud는 CNCF의 인큐베이팅 프로젝트로 합류했습니다.
*   Spectro Cloud는 Kubernetes 관리 솔루션을 위해 시리즈 C 펀딩으로 7천5백만 달러를 유치했습니다.
*   Solo.io는 자사의 Gloo API Gateway를 CNCF에 기증할 예정입니다.

**참석자들의 통찰력 및 트렌드:**
참석자들은 이번 행사의 주요 목표가 동료 기여자들과의 연결(특히 대면 만남), 클라우드 네이티브 기술과의 AI/ML 통합 학습, 진화하는 보안 트렌드 이해, 그리고 성능 최적화 탐색이었다고 강조했습니다.

관찰된 주요 트렌드는 다음과 같습니다:
*   **AI/ML 통합:** AI 워크로드를 클라우드 네이티브 플랫폼, 특히 Kubernetes에 통합하는 데 상당한 초점이 맞춰졌습니다.
*   **보안:** 워크로드 강화, 종단 간(end-to-end) 보안, 취약점 관리 및 새로운 보안 도구에 대한 논의가 주된 주제였습니다. 이는 기조연설(2일차)에서도 반영된 테마였습니다.
*   **커뮤니티 및 협업:** 기여자 서밋 및 SIG 모임에서의 대면 상호작용, 연결 증진, 지식 공유의 가치가 강조되었습니다.
*   **성능 최적화:** 애플리케이션 성능을 측정하고 분석하는 도구에 대한 관심이 높았습니다.
*   **특정 기술:** Istio Ambient Mesh의 GA 달성과 Envoy 기반 애플리케이션 게이트웨이 사용 증가가 특히 언급되었습니다.